In [2]:
import glob
import os
import pandas as pd
import sqlite3
#print(sqlite3.version)

base="doge"
quote="usdt"
interval="1m"
enable_logging=True

work_dir = './'
csv_files = glob.glob(os.path.join(work_dir, f"{base.upper()}-{quote.upper()}_*_{interval}.csv"))
df = pd.concat((pd.read_csv(f) for f in csv_files), ignore_index=True)

pd.options.display.float_format = '{:.8f}'.format

#print(df.head(3))

if "base" not in df.columns:
    df.insert(1, "base", base.upper())
else:
    df["base"] = base.upper()

if "quote" not in df.columns:
    df.insert(2, "quote", quote.upper())
else:
    df["quote"] = quote.upper()

#print(df.head(3))
#print(df.columns)

db_name = f"{base.lower()}_kline.db"
table_name = f"{base.lower()}_{quote.lower()}_{interval}"
conn = sqlite3.connect(db_name)

conn.execute(f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    timestamp TEXT PRIMARY KEY,
    base TEXT,
    quote TEXT,
    open REAL,
    high REAL,
    low REAL,
    close REAL,
    volume REAL,
    close_time INTEGER,
    quote_asset_volume REAL,
    number_of_trades INTEGER,
    taker_buy_base_asset_volume REAL,
    taker_buy_quote_asset_volume REAL,
    ignore INTEGER
)
""")

for _, row in df.iterrows():
    conn.execute(f"""
    INSERT INTO {table_name} (
        timestamp, base, quote, open, high, low, close, volume, close_time,
        quote_asset_volume, number_of_trades, taker_buy_base_asset_volume,
        taker_buy_quote_asset_volume, ignore
    ) VALUES (
        ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?
    )
    ON CONFLICT(timestamp) DO UPDATE SET
        base=excluded.base,
        quote=excluded.quote,
        open=excluded.open,
        high=excluded.high,
        low=excluded.low,
        close=excluded.close,
        volume=excluded.volume,
        close_time=excluded.close_time,
        quote_asset_volume=excluded.quote_asset_volume,
        number_of_trades=excluded.number_of_trades,
        taker_buy_base_asset_volume=excluded.taker_buy_base_asset_volume,
        taker_buy_quote_asset_volume=excluded.taker_buy_quote_asset_volume,
        ignore=excluded.ignore
    """, tuple(row))

conn.commit()
conn.close()

num_lines = len(df)
report = f"Data saved to {db_name} in table {table_name} with {num_lines} lines extracted."

if enable_logging:
    print(report)

Data saved to doge_kline.db in table doge_usdt_1m with 89402 lines extracted.
